# CSCI 3155 Project 1 - Compile Arithmetic Expressions into Stack Machine Bytecode

*Note*: See [EnvironmentSetup](EnvironmentSetup.ipynb) notebook for instructions on how to set up sbt or IntelliJ and import the project.

## Instructions for Testing.

### Running the main program
The main function is defined in the file Main.scala The way you run this depends on whether you are using sbt alone or IntelliJ.

#### IntelliJ
Choose the "Run Main" configuration and click the play button to run:
![title](img/run_main.png)

#### sbt
 
You can run Main from the command prompt with the current directory as the very top directory of the project, 
by running the following commands:

```bash
$ sbt compile
$ sbt run
```

__TODO for Students:__ Can you write your own test cases inside the function main for the file Main.scala
Post these test cases for your fellow students to help them debug/test their code.

### Running ScalaTest tests

Also, we will use a powerful unit testing package called scalatest. The tests themselves are in two files in the directory
`src/test/scala/edu/colorado/csci3155/project1/`

#### IntelliJ
Choose the "Run Scala Tests" configuration (right below "Run Main" in the picture above) and click the play button to run

#### sbt
To run this go to the terminal and from the very top directory run:

```
$ sbt test
```

It will say success if all tests run or give you a failure messages.

## Instructions for submission
1. Ensure everything is saved.
2. Run the tests one last time to ensure everything works as expected. You will not be able to submit if your code does not at least compile.
3. Run the `checkAndZipSubmission` sbt task
    * (*sbt*) Run `$ sbt checkAndZipSubmission` in a terminal
    * (*IntelliJ*) Select the "Zip Submission" configuration from the dropdown and click the run button (play): ![title](img/run_zip.png)
4. Ensure whichever option you chose displayed `[success]` at the end
5. Upload the generated "submission.zip" file located at the project root folder (at the same level as "src", "build.sbt", etc.).

# The Language
We have encountered arithmetic expressions given by the grammar 


$$\begin{array}{rcll}
\textbf{Expr} & \rightarrow & Const(\textbf{Double}) \\
& | & Plus( \textbf{Expr}, \textbf{Expr})  \\
& | & Minus( \textbf{Expr}, \textbf{Expr}) \\
& | & Mult(\textbf{Expr}, \textbf{Expr}) \\
& | & Div(\textbf{Expr}, \textbf{Expr}) \\
& | & Log(\textbf{Expr}) \\
& | & Exp(\textbf{Expr}) \\
& | & Sine(\textbf{Expr}) \\
& | & Cosine(\textbf{Expr}) \\\\
\textbf{Double} & \rightarrow & \text{all double precision numbers in Scala}\\
\textbf{Identifier} & \rightarrow & [a-zA-Z][a-z\ A-Z\ 0-9\ \_]* & \text{Note: All strings that begin with letters}\\
&&& \text{a-z or A-Z and subsequently can contain a-z, A-Z, 0-9 or _ chars}
\end{array}$$

__Note__ No identifiers are included in this grammar. This is intentional.

The objective of this project is to explore compiling this into a stack machine bytecode and writing an emulator to execute this bytecode.

# Part 1: Stack Machine Bytecode

Imagine a stack data structure that contains double precision numbers and the following instructions that operate on this stack. Pay special attention to the `SubI` and `DivI` instructions since the order matters.

- `PushI(d)`: push the number d onto the stack.
- `PopI`: pop off the top element of the stack - throw an exception if the stack is empty.
- `AddI`: pop two numbers from the stack, add them and push the result back to the stack. Throw an exception if the stack is empty during any of the pop operations.
- `SubI`: pop two numbers from the stack: let the first number popped be `v1` and second number be `v2`, subtract them as `v2 - v1` (this order is very important) and push the result back to the stack. Throw an exception if the stack is empty during any of the pop operations.
- `MultI`: pop two numbers from the stack, multiply them and push the result back to the stack. Throw an exception if the stack is empty during any of the pop operations.
- `DivI`: pop two numbers from the stack, let the first number popped be `v1` and second number be `v2`, subtract them as `v2 / v1` (this order is very important)  and push the result back to the stack. Throw an exception if the stack is emptyduring any of the pop operations. Throw exception if division by zero.
- `LogI`: pop _one_ number from the stack, compute its log if positive and push the result back onto the stack. If non-positive throw an exception. Throw an exception if the stack is empty during any of the pop operations.
- `ExpI`: pop _one_ number from the stack, compute its exponential $e^x$ and push the result back onto the stack.  Throw an exception if the stack is empty during any of the pop operations
- `SineI/CosineI`: pop _one_ number from the stack, compute its sin/cos respectively, and push the result back onto the stack.  Throw an exception if the stack is empty during any of the pop operations.


Given a sequence of instructions (eg., `[ PushI(2.0), PushI(3.0), AddI, PushI(4.0), SubI ]`, we execute each instruction in turn starting from the empty stack. We will implement the stack as a list with the head of the list as the top of the stack.

- Initially the stack is empty. 
- When we execute  `PushI(2.0)`, the stack is `[ 2.0 ]`
- When we execute  `PushI(3.0)`, the stack is `[ 3.0, 2.0 ]`
- When we execute `AddI`, the stack becomes `[ 5.0 ]`
- When we execute `PushI(4.0)`, the stack becomes `[ 4.0, 5.0 ] `
- When we execute `SubI`, the stack becomes `[ 1.0 ]`

The final value computed is obtained by taking the top element of the stack after all instructions have executed and that is the value `1.0` in this example.

### Instructions for Part 1

Implement the methods `emulateSingleInstruction` and `emulateStackMachine` in the file `StackMachineEmulator.scala`. For your convenience, the stack machine instructions have been defined as a very simple inductive definition giving case classes for all the instructions. We will use an immutable List data structure to simulate the stack.

- `emulateSingleInstruction(stack, instruction)` takes in a stack  which is a list of Double, and instruction which is an instruction. It returns a stack of double resulting from executing that instructions.

- `emulateStackMachine(listOfInstructions)` asks you to return the final value computed by the listOfInstructions. You should call `emulateSingleInstruction` repeatedly. Use `foldLeft` over lists instead of a loop.

# Part 2: Compiling Expressions to a List of ByteCode Instructions

We will now describe the `compilation` of expressions into bytecode instructions.

For instance the expression `1.0 + (2.0 - 3.0) * 5.0` is represented as an AST

`Plus(Const(1.0), Mult(Minus(Const(2.0), Const(3.0)), Const(5.0))`

The overall goal of this part is to compile this down into a list of bytecode instructions that serves to evaluate this through the emulator you have built in part 1.

For example, the expression above produces the instructions

~~~

PushI(1.0)
PushI(2.0)
PushI(3.0)
MinusI
PushI(5.0)
MultI 
AddI

~~~

You should check that evaluating this sequence gives the result `-4.0`. Please pay particular attention to the order of the operands for `MinusI` according to the specification provided in Part 1.


The idea is to implement a function __compileExpr(e)__ that given an expression __e__ yields a _list of instructions_ according to the following operational semantics definition.

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\;\; (\textit{#3})} $$
$$\newcommand\comp{\textbf{compileExpr}}$$

### Constant Rule

The rule for constants is simple. An expression `Const(f)` compiles to the instruction `PushI(f)`.

$$\semRule{}{\comp(\texttt{Const(f)}) = [ \text{PushI}(f) ] }{const-rule}$$

Note again that $\comp$ maps expressions to _list_ of instructions.

### Add Rule

$$\semRule{\comp(\texttt{e1}) = L_1,\ \comp(\texttt{e2}) = L_2}{\comp(\texttt{Plus(e1, e2}) = ListContatenation(L_1, L_2 , [ AddI ]) }{add-rule}$$

The instructions concatenate the lists $L_1, L_2$ along with the list consisting of a single instruction `[ AddI ]`. Note that the `++` operator in scala implements the list concatenation.

### Subtract Rule

$$\semRule{\comp(\texttt{e1}) = L_1,\ \comp(\texttt{e2}) = L_2}{\comp(\texttt{Minus(e1, e2}) = ListContatenation(L_1, L_2 , [ SubI ]) }{minus-rule}$$

The instructions concatenate the lists $L_1, L_2$ along with the list consisting of a single instruction `[ SubI ]`. 


### Rules for Other expressions

We hope that you will be able to fill in rules for other cases `Mult`, `Div`, `Exp`, `Log`, `Sine` and `Cosine`.



### Instructions for Part 2

The definition of Expression AST is given in the file `Expr.scala`

Your goal is to implement the compilation routine `compileToStackMachineCode(e: Expr): List[StackMachineInstruction]` in the file `StackMachineCompilation.scala`. The function takes in an expression `e` and outputs a list of stack machine instructions.